# 🧪 TPC-DS Benchmark Queries 16–20
Databricks SQL using `%run ./specify_catalog_schema`

In [0]:
%run ./specify_catalog_schema

In [0]:
%sql
-- Query 16: Catalog Sales Shipping and Profit Summary
-- TPC-DS Query 16
SELECT
    COUNT(DISTINCT cs_order_number) AS order_count,
    SUM(cs_ext_ship_cost) AS total_shipping_cost,
    SUM(cs_net_profit) AS total_net_profit
FROM catalog_sales cs1
JOIN date_dim ON cs1.cs_ship_date_sk = d_date_sk
JOIN customer_address ON cs1.cs_ship_addr_sk = ca_address_sk
JOIN call_center ON cs1.cs_call_center_sk = cc_call_center_sk
WHERE d_date BETWEEN '2002-03-01' AND DATE_ADD('2002-03-01', 60)
  AND ca_state = 'IA'
  AND cc_county IN ('Williamson County', 'Williamson County', 'Williamson County', 'Williamson County', 'Williamson County')
  AND EXISTS (
      SELECT 1
      FROM catalog_sales cs2
      WHERE cs1.cs_order_number = cs2.cs_order_number
        AND cs1.cs_warehouse_sk <> cs2.cs_warehouse_sk
  )
  AND NOT EXISTS (
      SELECT 1
      FROM catalog_returns cr1
      WHERE cs1.cs_order_number = cr1.cr_order_number
  );

In [0]:
%sql
-- Query 17: Top M Returns with Reason
-- TPC-DS Query 17
SELECT
    i_item_id,
    i_item_desc,
    i_category,
    i_class,
    i_current_price,
    SUM(ss_ext_sales_price) AS itemrevenue,
    SUM(ss_ext_sales_price) * 100 / SUM(SUM(ss_ext_sales_price)) OVER (PARTITION BY i_class) AS revenueratio
FROM store_sales
JOIN item ON ss_item_sk = i_item_sk
JOIN date_dim ON ss_sold_date_sk = d_date_sk
WHERE d_year = 2000
  AND d_moy = 2
GROUP BY
    i_item_id,
    i_item_desc,
    i_category,
    i_class,
    i_current_price
ORDER BY
    i_category,
    i_class,
    i_item_id,
    i_item_desc,
    revenueratio
LIMIT 100;

In [0]:
%sql
-- Query 18: Average Price Per Category
-- TPC-DS Query 18
SELECT
    i_item_id,
    ca_country,
    ca_state,
    ca_county,
    AVG(cs_quantity) AS agg1,
    AVG(cs_list_price) AS agg2,
    AVG(cs_coupon_amt) AS agg3,
    AVG(cs_sales_price) AS agg4,
    AVG(cs_net_profit) AS agg5,
    AVG(c_birth_year) AS agg6,
    AVG(cd1.cd_dep_count) AS agg7
FROM catalog_sales
JOIN customer_demographics cd1 ON cs_bill_cdemo_sk = cd1.cd_demo_sk
JOIN customer_demographics cd2 ON cs_ship_cdemo_sk = cd2.cd_demo_sk
JOIN customer ON cs_ship_customer_sk = c_customer_sk
JOIN customer_address ON c_current_addr_sk = ca_address_sk
JOIN date_dim ON cs_sold_date_sk = d_date_sk
JOIN item ON cs_item_sk = i_item_sk
WHERE cd1.cd_gender = 'F'
  AND cd1.cd_education_status = 'College'
  AND cd2.cd_gender = 'F'
  AND cd2.cd_education_status = 'College'
  AND c_birth_month IN (1, 6, 12)
  AND d_year = 2000
GROUP BY
    i_item_id,
    ca_country,
    ca_state,
    ca_county
ORDER BY
    ca_country,
    ca_state,
    ca_county,
    i_item_id
LIMIT 100;

In [0]:
%sql
-- Query 19: Sales and Returns by Category and Brand
-- TPC-DS Query 19
SELECT
    i_brand_id,
    i_brand,
    i_manufact_id,
    i_manufact,
    SUM(ss_ext_sales_price) AS ext_price
FROM store_sales
JOIN item ON ss_item_sk = i_item_sk
JOIN date_dim ON ss_sold_date_sk = d_date_sk
JOIN customer ON ss_customer_sk = c_customer_sk
JOIN customer_address ON c_current_addr_sk = ca_address_sk
JOIN store ON ss_store_sk = s_store_sk
WHERE i_manager_id = 7
  AND d_moy = 11
  AND d_year = 1999
  AND substr(ca_zip, 1, 5) <> substr(s_zip, 1, 5)
GROUP BY
    i_brand,
    i_brand_id,
    i_manufact_id,
    i_manufact
ORDER BY
    ext_price DESC,
    i_brand,
    i_brand_id,
    i_manufact_id,
    i_manufact
LIMIT 100;

In [0]:
%sql
-- Query 20: Top Customers by Sales Amount
-- TPC-DS Query 20
SELECT
    c_customer_id,
    c_first_name,
    c_last_name,
    SUM(ss_net_paid) AS total_sales
FROM store_sales
JOIN customer ON ss_customer_sk = c_customer_sk
JOIN date_dim ON ss_sold_date_sk = d_date_sk
WHERE d_year = 2001
GROUP BY
    c_customer_id,
    c_first_name,
    c_last_name
ORDER BY
    total_sales DESC
LIMIT 20;